In [22]:
import time
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import dgl
from dgl.data import CoraGraphDataset, CiteseerGraphDataset, PubmedGraphDataset
from models import DGC
from time import perf_counter

In [23]:
lr = 0.2
bias = False
epochs = 50
weight_decay = 5e-4
loss_fn = torch.nn.CrossEntropyLoss()

In [24]:
data = PubmedGraphDataset()
g = data[0]
features = g.ndata['feat']
labels = g.ndata['label']
train_mask = g.ndata['train_mask']
val_mask = g.ndata['val_mask']
test_mask = g.ndata['test_mask']
in_feats = features.shape[1]
n_classes = data.num_labels
n_edges = g.number_of_edges()
g = dgl.remove_self_loop(g)
g = dgl.add_self_loop(g)

  NumNodes: 19717
  NumEdges: 88651
  NumFeats: 500
  NumClasses: 3
  NumTrainingSamples: 60
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


In [25]:
model = DGC(in_feats, n_classes, K=2, T=2, bias=bias)
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

In [26]:
def test(model, g, features, labels, mask):
    model.eval()
    with torch.no_grad():
        logits = model(g, features)[mask]  # only compute the evaluation set
        labels = labels[mask]
        _, indices = torch.max(logits, dim=1)
        correct = torch.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)

In [27]:
def train(model, g, features, labels, train_mask, val_mask, test_mask):
    t = perf_counter()
    for epoch in range(epochs):
        model.train()
        logits = model(g, features)
        loss = loss_fn(logits[train_mask], labels[train_mask])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        model.eval()
        logits = model(g, features)
        train_acc = torch.sum(logits[train_mask].argmax(1) == labels[train_mask]).item() / train_mask.sum().item()
        val_acc = torch.sum(logits[val_mask].argmax(1) == labels[val_mask]).item() / val_mask.sum().item()
        test_acc = torch.sum(logits[test_mask].argmax(1) == labels[test_mask]).item() / test_mask.sum().item()
        print(f'Epoch {epoch + 1:02d}, Loss: {loss:.4f}, Train: {train_acc:.4f}, Val: {val_acc:.4f}, Test: {test_acc:.4f}')

    training_time = perf_counter()-t
    print(f'Training time: {training_time:.4f}s')

In [28]:
train(model, g, features, labels, train_mask, val_mask, test_mask)
test(model, g, features, labels, test_mask)

Epoch 01, Loss: 1.0973, Train: 0.9333, Val: 0.7040, Test: 0.7150
Epoch 02, Loss: 1.0072, Train: 0.9333, Val: 0.7220, Test: 0.7320
Epoch 03, Loss: 0.9295, Train: 0.9500, Val: 0.7380, Test: 0.7380
Epoch 04, Loss: 0.8656, Train: 0.9667, Val: 0.7440, Test: 0.7430
Epoch 05, Loss: 0.8155, Train: 0.9667, Val: 0.7440, Test: 0.7430
Epoch 06, Loss: 0.7774, Train: 0.9667, Val: 0.7460, Test: 0.7460
Epoch 07, Loss: 0.7495, Train: 0.9667, Val: 0.7460, Test: 0.7430
Epoch 08, Loss: 0.7298, Train: 0.9667, Val: 0.7540, Test: 0.7450
Epoch 09, Loss: 0.7168, Train: 0.9667, Val: 0.7540, Test: 0.7480
Epoch 10, Loss: 0.7089, Train: 0.9667, Val: 0.7600, Test: 0.7550
Epoch 11, Loss: 0.7048, Train: 0.9667, Val: 0.7680, Test: 0.7520
Epoch 12, Loss: 0.7034, Train: 0.9500, Val: 0.7700, Test: 0.7570
Epoch 13, Loss: 0.7040, Train: 0.9500, Val: 0.7760, Test: 0.7630
Epoch 14, Loss: 0.7057, Train: 0.9500, Val: 0.7780, Test: 0.7640
Epoch 15, Loss: 0.7081, Train: 0.9500, Val: 0.7740, Test: 0.7630
Epoch 16, Loss: 0.7107, T

0.771